In [1]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import shutil
from PIL import ImageFont, ImageDraw, Image

In [4]:
# 이미지 전처리 하기 전 폴더 내에서 전처리 안된 이미지 확인

lst = []
for i in os.listdir('E:\\food_img2000\\train\\train'):
    if i not in os.listdir('E:\\img'):
        lst += [i]

In [2]:
# yolo 모델을 학습하기 위해서는 data.yaml 파일이 필요한데 이 파일 내에 레이블에 대한 내용이 들어있어야 함
# 일일이 다 입력하기 귀찮은 관계로 딕셔너리 작업 수행

dic = {}
x = 0

for food in os.listdir('E:\img'):
    dic[food] = x
    x += 1

dic

{'albab': 0,
 'bibimbab': 1,
 'bibimnangmyeon': 2,
 'blackkongjorim': 3,
 'budaejjigae': 4,
 'bulgogi': 5,
 'chinamul': 6,
 'chongakgimchi': 7,
 'creampasta': 8,
 'dakkalguksu': 9,
 'danjangjjigae': 10,
 'donggrangttang': 11,
 'dongtaejun': 12,
 'donkkasu': 13,
 'dububuchim': 14,
 'eggguk': 15,
 'eggjangjorim': 16,
 'eggjjim': 17,
 'eggmari': 18,
 'emukbokkum': 19,
 'emukguk': 20,
 'figgimchijjigae': 21,
 'galchijorim': 22,
 'gamjabokkum': 23,
 'ggatipjjangajji': 24,
 'gimbab': 25,
 'gimbokbab': 26,
 'gimchiguk': 27,
 'gimchijun': 28,
 'gimchimariguksu': 29,
 'gogimandu': 30,
 'gondrebab': 31,
 'gosarinamul': 32,
 'gunsaeubokkum': 33,
 'haemulpajun': 34,
 'hambaksteak': 35,
 'hambuchim': 36,
 'hobakjun': 37,
 'huenangmyeon': 38,
 'hunjeduck': 39,
 'janchiguksu': 40,
 'jinmichaebokkum': 41,
 'jjajangmyeon': 42,
 'jjamfong': 43,
 'jjimdak': 44,
 'jjolmyeon': 45,
 'jokbal': 46,
 'kongguksu': 47,
 'makguksu': 48,
 'memilguksu': 49,
 'miyeokguk': 50,
 'mukmuchim': 51,
 'mulchibookum': 52,
 

### 1. 이미지 전처리

In [ ]:
for food_name in lst: # 음식 사진 들어있는 폴더 이름

    print(food_name)
    
    model = YOLO('yolov8s.pt')

    folder_name = f'E:\\food_img2000\\train\\train\\{food_name}' # 크롭할 이미지가 있는 폴더

    file_num = 1

    os.makedirs(f'E:\img\\{food_name}') # 전처리한 이미지를 저장할 폴더 생성

    for file_name in os.listdir(f'{folder_name}'):
        img_name = f'{folder_name}\\{file_name}'
        img = cv2.imread(img_name)
        img = cv2.resize(img,(640,640)) # YOLOv8의 기본 이미지 크기에 맞게 이미지 사이즈 재설정
        results = model.predict(img)

        try:
            a, b, c, d = [int(i) for i in (results[0].boxes.xyxy[0])]
            
            x1, y1 = a, b
            x2, y2 = c, d

            x1, x2 = min(a, c), max(a, c)
            y1, y2 = min(b, d), max(b, d)

            cropped_img = img[y1+2:y2-2, x1+2:x2-2] # 이미지 내 객체만 추출
            cropped_img_path = f'E:\img\{food_name}\\{food_name}_{file_num}.jpg' # 크롭한 이미지가 저장될 폴더 + 이미지 이름

            cv2.imwrite(cropped_img_path, cropped_img) # 크롭한 이미지 저장
            file_num += 1
        except:
            continue

### 2. yolov8 모델을 학습시키기 위해 이미지를 다른 폴더로 이동

In [6]:
# 기존 폴더에서 다른 폴더로 이미지를 이동
import os
import random
import shutil

for food in os.listdir('E:\img'):

# 원본 폴더 경로
    source_folder = f'E:\\img\\{food}'
    # 대상 폴더 경로
    destination_folder = f'E:\yolov8\datasets\\train\\images'

    # 원본 폴더에서 파일 목록 가져오기
    all_files = os.listdir(source_folder)

    selected_files = all_files[:600]

    # 선택한 파일을 대상 폴더로 이동
    for file_name in selected_files:
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        shutil.copy(source_path, destination_path)
        f = file_name.split('.')[0] + '.txt' 
        with open(f'E:\yolov8\datasets\\train\labels\\{f}','w+') as file:
            file.write(' '.join(list(map(str,[dic[food], 0.5, 0.5, 1,1]))))

### 3. yaml 파일 생성하기

In [ ]:
import yaml

data = {
    "train" : '/datasets/train/',
        "val" : '/datasets/val/',
        "names" : dic}

with open('./data.yaml', 'w') as f :
    yaml.dump(data, f)

with open('./data.yaml', 'r') as f :
    lines = yaml.safe_load(f)
    print(lines)

### 4. 모델 학습하기

- mAP@0.5에서 91.1%의 성능 달성

In [ ]:
model = YOLO('yolov8s.pt') # 사전학습된 모델 로드

In [ ]:
import torch

# cuda memory 비우기 -> 메모리에 쌓이면 모델을 학습할 수가 없음
torch.cuda.empty_cache()
torch.cuda.memory_summary()

In [ ]:
model.train(
  data='data.yaml',
  epochs=12,     # 더 많은 에포크로 학습
  patience=5,    # 성능 개선이 없는 에포크 수를 늘려 더 많은 기회 제공
  batch=64     # 배치 크기를 늘려 학습 속도 향상 -> 32로 하면 학습속도가 느려짐
)

In [ ]:
model.save('model_last.pt')

### 5. 모델을 기반으로 예측 수행하기

- 기존 유사 서비스에 대한 차별성을 위해 당뇨병 환자에게 좋지 않은 음식인 경우 붉은색 글씨로 사진 위에 레이블 표기

In [1]:
model = YOLO('model_last.pt') # 모델 불러오기 
image = cv2.imread("./test1.png", cv2.IMREAD_COLOR)
image = cv2.resize(image,(640,640))
result = model.predict(source=image)[0] 
label = result.boxes.cls # 모델이 예측한 레이블 결과 ex) 30, 11, 2, 15
warning = [8,13] # 당뇨병 환자에게 위험한 음식의 레이블
fontpath = "E:\\yolov8\\Pretendard-Bold.ttf"
font = ImageFont.truetype(fontpath, 30)

for i in range(len(result.boxes.xyxy)):
    x = int((result.boxes.xyxy[i][0] + result.boxes.xyxy[i][2]) / 2) # 인식한 객체의 바운딩 박스 가로 정가운데
    y = int((result.boxes.xyxy[i][1] + result.boxes.xyxy[i][3]) / 2) # 인식한 객체의 바운딩 박스 세로 정가운데

    pred_result = result.names[int(label[i])] # 음식 목록에서 모델이 예측한 레이블에 해당하는 값 가져오기
    # ex) 미역국

    overlay = image.copy()
    left, top, right, bottom = font.getbbox(pred_result)
    width = right - left
    height = bottom - top
    text_size = [width,height] # 텍스트 가로, 세로 길이

    cv2.rectangle(overlay, (x-(text_size[0]//2 + 5),y-(text_size[1]//2 + 5)), (x+(text_size[0]//2 + 5),y+(text_size[1]//2 + 10)), (0, 0, 0), -1)  # 검정색 배경
    alpha = 0.6  # 투명도 설정
    # 텍스트 크기 기반으로 텍스트에 맞춰서 불투명 사각형 생성
    cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)
    
    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # OpenCV 이미지를 PIL 이미지로 변환
    draw = ImageDraw.Draw(img_pil)
    if label[i] in warning: # 만약 음식을 인식했는데 당뇨병 환자에게 좋지 않은 음식이라면
        draw.text((x - (text_size[0] // 2), y - (text_size[1] // 2)), pred_result, font=font, fill=(255, 0, 0))
        # 붉은 색으로 한글 텍스트 표시
    else:
        draw.text((x - (text_size[0] // 2), y - (text_size[1] // 2)), pred_result, font=font, fill=(255, 255, 255)) 


    # PIL 이미지를 다시 OpenCV 이미지로 변환
    image = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    
cv2.imshow('draw',image)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 640x640 1 , 907.0ms
Speed: 16.0ms preprocess, 907.0ms inference, 18.0ms postprocess per image at shape (1, 3, 640, 640)
